## Pulse-echo ultrasound attenuation tomography

This notebook shows how to image the spatial distribution of tissue attenuation with the pulse-echo ultrasound attenuation tomography technique presented in [tba]. Only linear probes are supported in this version. 

Note: We do not perform ensemble averaging over different realizations to make this notebook easy to follow and understand. However, this averaging can be easily included following the approach in 'Compute_calibration_data.ipynb'.

Author: Naiara Korta Martiartu (naiara.korta@unibe.ch)\
Date: Feb. 2024

In [ ]:
# Import useful packages

%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pickle
from attomo import Bmode, ATTomo

matplotlib.rcParams.update({'font.size': 18})

#### Initialize Bmode object

In [ ]:
rec = Bmode('kWave')

#### Load data and acquisition parameters

In [ ]:
# Define path to the folder and the filename containing data
folder_path = './data/'
data_filename   = 'acquisition_1.mat'

# load data
rec.load_data(folder_path, data_filename, use_filter=True) 

#### Load calibration data

In [ ]:
folder = './data/'
filename   = 'calibration.pkl'

with open(folder + filename,'rb') as f:  
    calibration = pickle.load(f)

### Step 1: Delay and sum beamforming 

In [ ]:
rec.acq_delay = rec.acq_delay + 6.954878861969333e-07 # fix acquisition delay so that t=0 is at max. of the pulse

rec.das(c = 1540, rec_range = 37.5e-3, gpu=True)

#### Plot compounded B-mode image

In [ ]:
rec.plot_bmode(option_rec=0, dym_range=60, normlevel=-30, option_comp=2, savefig=False)

### Step 2: Synthetic focusing in emission

In [ ]:
# Define synthetic aperture angles
start_saangle = -25  # [deg]
end_saangle = 25
delta_saangle = 2.5

sa_angles = np.arange(start_saangle, end_saangle + delta_saangle, delta_saangle) * np.pi / 180  # [rad]


# Define standard deviation for Gaussian weighting of images 
sa_radius = 3
sa_sigma=sa_radius*np.pi/180/np.sqrt(2)  # [rad]


# Coherent compounding to reduce clutter 
rec.coherent_compounding(sa_angles=sa_angles, sa_sigma=sa_sigma)

### Step 3: Compute normalized cross-correlation log-amplitude measurements

#### Initialize ATTomo object

In [ ]:
tomo = ATTomo(rec, spacing=[0.5e-3, 0.5e-3], gpu=False, range_rec=37.5e-3)

#### Measure data

In [ ]:
tomo.logamp_measurements(kernel_size=[1e-3, 1e-3])

#### Define relevant angles for tomography, add data and interpolate to tomography grid 

In [ ]:
start_angle = -25  # [deg]
end_angle = 25
delta_angle = 2.5

tomo_angles = np.arange(start_angle,end_angle + delta_angle,delta_angle)*np.pi/180  # [deg]


tomo.add_data(tomo_angles)

# save for later
tmp = tomo.tomo_logamp_map 

#### Calibrate data

In [ ]:
tomo.tomo_logamp_map = tmp - calibration

#### Plot data

In [ ]:
fig = plt.figure(figsize=(15,12))

# Define extent of imaging domain
extent = 100 * np.min(tomo.tomo_x), 100 * np.max(tomo.tomo_x), 100 * np.max(tomo.tomo_z), 100 * np.min(tomo.tomo_z)

        
for i in range(tomo.tomo_nangles - 1):
    
    ax = fig.add_subplot(4, 5, i+1)    
    
    im = ax.imshow(tomo.tomo_logamp_map[:,:,i], aspect = 'equal', extent = extent,
                     vmin=-0.2, vmax=0.2, cmap='RdBu')
    
    
    
fig.colorbar(im)

### Step 4: Reconstruct spatial distribution of tissue attenuation

#### Compute elements of the forward operator

In [ ]:
tomo.compute_forward_op()

#### Plot ray density

In [ ]:
tomo.plot_ray_density()

#### Compute L-curve to optimize regularization parameter values

In [ ]:
tomo.plot_lcurve(order=1)

#### Reconstruct attenuation images

In [ ]:
reg_param = 5e-6  # regularization parameter value 
order = 1  # order Tikhonov regularization


## Run inversion and compute misfit reduction
tomo.inversion(reg_parameter=[reg_param, reg_param], order=order, att_exponent=1.0)  
tomo.misfit_reduction() 

#### Plot attenuation image

In [ ]:
# 

plt.imshow(np.reshape(tomo.model_rec, (tomo.nz, tomo.nx)), aspect = 'equal', extent = extent,
           vmin=0, vmax=50, cmap='viridis')

plt.colorbar(ticks=[0, 50]).set_label('Attenuation \n (Np/m)')
plt.xlabel('x (cm)')
plt.ylabel('z (cm)')

#### Plot diagonal posterior covariance

In [ ]:
plt.imshow(np.reshape(np.diag(tomo.post)/max(np.diag(tomo.post)), (tomo.nz, tomo.nx)), 
           aspect = 'equal', extent = extent, vmin=0, vmax=0.5, cmap='plasma')

plt.colorbar(ticks=[0, 0.5]).set_label('Norm. variance')
plt.xlabel('x (cm)')
plt.ylabel('z (cm)')

### Step 5: Check data misfit

#### Compute predicted data from reconstructed model

In [ ]:
pred_data=tomo.compute_obs_data(tomo.model_rec)

#### Plot discrepancies between observed and predicted data

In [ ]:
fig = plt.figure(figsize=(15,12))
        
for i in range(tomo.tomo_nangles - 1):
    
    ax = fig.add_subplot(4, 5, i+1) 
    
    # note: obs_data is -tomo.tomo_logamp_map (with negative sign) 
    im = ax.imshow(-tomo.tomo_logamp_map[:,:,i] - pred_data[:,:,i], aspect = 'equal', extent = extent,
                     vmin=-0.2, vmax=0.2, cmap='RdBu')
    
    
    
fig.colorbar(im)